# Application Project Milestone 2

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
from scipy.stats import ttest_rel

# ================
# Step 0: 读取 CSV
# ================
X_train_raw = pd.read_csv('./Raw_features/X_train.csv')
y_train_raw = pd.read_csv('./Raw_features/y_train.csv')
X_test_raw  = pd.read_csv('./Raw_features/X_test.csv')
y_test_raw  = pd.read_csv('./Raw_features/y_test.csv')

X_train_handle = pd.read_csv('./Handle_selected_features/X_train.csv')
y_train_handle = pd.read_csv('./Handle_selected_features/Y_train.csv')
X_test_handle  = pd.read_csv('./Handle_selected_features/X_test.csv')
y_test_handle  = pd.read_csv('./Handle_selected_features/Y_test.csv')

X_train_cluster = pd.read_csv('./Cluster_based_features/X_cluster_train.csv')
y_train_cluster = pd.read_csv('./Cluster_based_features/Y_cluster_train.csv')
X_test_cluster  = pd.read_csv('./Cluster_based_features/X_cluster_test.csv')
y_test_cluster  = pd.read_csv('./Cluster_based_features/Y_cluster_test.csv')

X_train_pca_2 = pd.read_csv('./PCA_transformed_features/X_train_pca.csv')
y_train_pca_2 = pd.read_csv('./PCA_transformed_features/y_train_pca.csv')
X_test_pca_2  = pd.read_csv('./PCA_transformed_features/X_test_pca.csv')
y_test_pca_2  = pd.read_csv('./PCA_transformed_features/y_test_pca.csv')

X_train_pca_opt = pd.read_csv('./PCA_transformed_features/X_train_pca_opt.csv')
y_train_pca_opt = pd.read_csv('./PCA_transformed_features/y_train_pca_opt.csv')
X_test_pca_opt  = pd.read_csv('./PCA_transformed_features/X_test_pca_opt.csv')
y_test_pca_opt  = pd.read_csv('./PCA_transformed_features/y_test_pca_opt.csv')

# 通常 y_* CSV 只有标签，可以 .values.ravel() 转成 ndarray
y_train_raw = y_train_raw.values.ravel()
y_test_raw  = y_test_raw.values.ravel()

y_train_handle = y_train_handle.values.ravel()
y_test_handle  = y_test_handle.values.ravel()

y_train_cluster = y_train_cluster.values.ravel()
y_test_cluster  = y_test_cluster.values.ravel()

y_train_pca_2 = y_train_pca_2.values.ravel()
y_test_pca_2  = y_test_pca_2.values.ravel()

y_train_pca_opt = y_train_pca_opt.values.ravel()
y_test_pca_opt  = y_test_pca_opt.values.ravel()

# 同理，如果 X_* CSV 只有特征，通常可以 .values 转成 ndarray
X_train_raw = X_train_raw.values
X_test_raw  = X_test_raw.values

X_train_handle = X_train_handle.values
X_test_handle  = X_test_handle.values

X_train_cluster = X_train_cluster.values
X_test_cluster  = X_test_cluster.values

X_train_pca_2 = X_train_pca_2.values
X_test_pca_2  = X_test_pca_2.values

X_train_pca_opt = X_train_pca_opt.values
X_test_pca_opt  = X_test_pca_opt.values

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =========================
# Step 1: 定义一个简单的ANN
# =========================
class RegressionANN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.layer(x)


# ======================================================
# Step 2: 定义训练 + 验证（10-fold CV）的函数 (PyTorch 版本)
# ======================================================
def ann_train_and_evaluate(X, y, n_splits=10, n_epochs=100, batch_size=32, lr=0.001):
    """
    对 (X, y) 做 n_splits 折交叉验证，返回各 fold 的验证集 MSE。
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_errors = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
        y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1).to(device)

        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        model = RegressionANN(X.shape[1]).to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        # 训练模型
        for epoch in range(n_epochs):
            model.train()
            for xb, yb in train_loader:
                preds = model(xb)
                loss = criterion(preds, yb)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        # 验证
        model.eval()
        with torch.no_grad():
            val_preds = model(X_val_tensor)
            val_loss = criterion(val_preds, y_val_tensor).item()
            fold_errors.append(val_loss)

    return fold_errors


# =========================================
# Step 3: 对五类特征做 10-fold CV 并记录误差
# =========================================
errors_baseline   = ann_train_and_evaluate(X_train_raw,       y_train_raw)
errors_handle     = ann_train_and_evaluate(X_train_handle,    y_train_handle)
errors_cluster    = ann_train_and_evaluate(X_train_cluster,   y_train_cluster)
errors_pca_2      = ann_train_and_evaluate(X_train_pca_2,     y_train_pca_2)
errors_pca_opt    = ann_train_and_evaluate(X_train_pca_opt,   y_train_pca_opt)

# 把各自的误差保存到一个 dict 里方便后续处理
models_errors = {
    "baseline_raw"   : errors_baseline,
    "hand_selected"  : errors_handle,
    "cluster_based"  : errors_cluster,
    "pca_2"          : errors_pca_2,
    "pca_opt"        : errors_pca_opt
}


# ======================================
# Step 4: 查看平均 MSE，找出最佳模型
# ======================================
for name, errs in models_errors.items():
    mean_mse = np.mean(errs)
    std_mse  = np.std(errs)
    print(f"{name} => mean MSE: {mean_mse:.4f}, std: {std_mse:.4f}")

# 比如可以用最小 mean MSE 为“最佳”
best_model_name = min(models_errors, key=lambda k: np.mean(models_errors[k]))
best_model_errs = models_errors[best_model_name]

print(f"\n=> Best model by CV mean MSE: {best_model_name}, MSE = {np.mean(best_model_errs):.4f}")


# =====================================================
# Step 5: 对 “baseline_raw” 与“最佳模型”做配对 t 检验
# =====================================================
if best_model_name != "baseline_raw":
    baseline_errs = models_errors["baseline_raw"]
    t_stat, p_value = ttest_rel(baseline_errs, best_model_errs)

    print(f"\nPaired t-test: baseline_raw vs {best_model_name}")
    print(f"t-statistic = {t_stat:.4f}, p-value = {p_value:.4f}")

    alpha = 0.05
    if p_value < alpha:
        print("=> 差异显著: 最佳模型显著优于 baseline_raw")
    else:
        print("=> 差异不显著: 最佳模型不显著优于 baseline_raw")
else:
    print("\n最优模型居然是 baseline_raw，本身就无从和自己比了。")


baseline_raw => mean MSE: 0.0343, std: 0.0041
hand_selected => mean MSE: 0.0675, std: 0.0053
cluster_based => mean MSE: 0.0941, std: 0.0042
pca_2 => mean MSE: 0.0906, std: 0.0063
pca_opt => mean MSE: 0.0358, std: 0.0027

=> Best model by CV mean MSE: baseline_raw, MSE = 0.0343

最优模型居然是 baseline_raw，本身就无从和自己比了。
